<a href="https://colab.research.google.com/github/hws2002/MachineLearning_PytorchNScikitLearn/blob/master/chapter15_2_RNN_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 15.2 시퀀스 모델링을 위한 RNN


실제로 어떻게 동작하는지 보기 위해 은닉-은닉 순환의 정방향 계산을 수동으로 수행해 보자.  
torch.nn모듈의 RNN클래스로 순환 층을 만들고 길이가 3인 입력 시퀀스에서 정방향 계산을 수행하여 출력을 만들자.  
그 후 수동으로 정방향 계산을 수행하여 RNN의 결과와 비교해 보자

In [1]:
import torch
import torch.nn as nn
torch.manual_seed(1)
rnn_layer = nn.RNN(input_size = 5, hidden_size = 2, num_layers = 1, batch_first  = True)
# batch_first 를 True로 지정했기 때문에, 입력 크기는 (batch_size, sequence_length, 5)가 된다
# 이중 5는 input_size가 5이기 때문이다
# 여러 개의 RNN 층을 쌓으려면 num_layers 매개변수를 1 이상으로 지정하면 된다.
rnn_layer

RNN(5, 2, batch_first=True)

In [2]:
w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh 크기:', w_xh.shape)
print('W_hh 크기:', w_hh.shape)
print('b_xh 크기:', b_xh.shape)
print('b_xh 크기:', b_hh.shape)


W_xh 크기: torch.Size([2, 5])
W_hh 크기: torch.Size([2, 2])
b_xh 크기: torch.Size([2])
b_xh 크기: torch.Size([2])


In [5]:
# rnn_layer의 정방향 계산을 수행하고 각 타임 스텝에서 수동으로 출력을 계산하여 비교해 보자
x_seq = torch.tensor([
  [1.0]*5,
  [2.0]*5,
  [3.0]*5
]).float()

# 간단한 RNN의 출력
output, hn = rnn_layer(torch.reshape(x_seq,(1,3,5)))
out_man = []

# 수동으로 출력 계산하기:
for t in range(3):
  xt = torch.reshape(x_seq[t], (1,5))
  print(f'타임 스탭 {t} =>')
  print('   입력          :', xt.numpy())

  ht = torch.matmul(xt, torch.transpose(w_xh,0,1)) + b_xh # w_xh의 첫번째 차원과 두번째 차원을 교환
  print('   은닉          :', ht.detach().numpy())
  if t > 0:
    prev_h = out_man[t-1]
  else :
    prev_h = torch.zeros((ht.shape))
  ot = ht + torch.matmul( prev_h, torch.transpose(w_hh,0,1)) + b_hh
  ot = torch.tanh(ot) # RNN에서 기본값으로 하이퍼볼릭 탄젠트 활성화 함수를 씀. RNN의 nonlinearity 매개변수로 다른 활성화 함수를 지정할 수 있음
  out_man.append(ot)
  print(' 출력 (수동) : ', ot.detach().numpy())
  print(' RNN 출력  :', output[:,t].detach().numpy())
  print()

타임 스탭 0 =>
   입력          : [[1. 1. 1. 1. 1.]]
   은닉          : [[-0.4701929   0.58639044]]
 출력 (수동) :  [[-0.3519801   0.52525216]]
 RNN 출력  : [[-0.3519801   0.52525216]]

타임 스탭 1 =>
   입력          : [[2. 2. 2. 2. 2.]]
   은닉          : [[-0.88883156  1.2364398 ]]
 출력 (수동) :  [[-0.68424344  0.76074266]]
 RNN 출력  : [[-0.68424344  0.76074266]]

타임 스탭 2 =>
   입력          : [[3. 3. 3. 3. 3.]]
   은닉          : [[-1.3074702  1.8864892]]
 출력 (수동) :  [[-0.8649416  0.9046636]]
 RNN 출력  : [[-0.8649416  0.9046636]]

